# AIML Online Capstone - AUTOMATIC TICKET ASSIGNMENT

Pre-Processing, Data Visualization and EDA
● Exploring the given Data files
● Understanding the structure of data
● Missing points in data
● Finding inconsistencies in the data
● Visualizing different patterns
● Visualizing different text features
● Dealing with missing values
● Text preprocessing
● Creating word vocabulary from the corpus of report text data
● Creating tokens as required

In [0]:
# Initialize the random number generator
import random
random.seed(0)

# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
#Set your project path 
import os
project_path =os.chdir('/content/drive/My Drive/Colab Notebooks/Capstone') 

In [4]:

import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
#from tensorflow.contrib.rnn import BasicLSTMCell
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn
import time
import os
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
df = pd.read_excel (r'input_data.xlsx')

In [6]:
df.head()

,Short description,Description,Caller,Assignment group
0,login issue,-verified user details.(employee# & manager na...,spxjnwir pjlcoqds,GRP_0
1,outlook,\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...,hmjdrvpb komuaywn,GRP_0
2,cant log in to vpn,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail...,eylqgodm ybqkwiam,GRP_0
3,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0


In [7]:
df.describe()

,Short description,Description,Caller,Assignment group
count,8492,8499,8500,8500
unique,7481,7817,2950,74
top,password reset,the,bpctwhsn kzqsbmtp,GRP_0
freq,38,56,810,3976


In [8]:
df.isnull().sum()

Short description    8
Description          1
Caller               0
Assignment group     0
dtype: int64

In [0]:
nan_rows1 = df[df['Short description'].isnull()]
nan_rows2 = df[df['Description'].isnull()]

In [10]:
nan_rows1

,Short description,Description,Caller,Assignment group
2604,NaN,\r\n\r\nreceived from: ohdrnswl.rezuibdt@gmail...,ohdrnswl rezuibdt,GRP_34
3383,NaN,\r\n-connected to the user system using teamvi...,qftpazns fxpnytmk,GRP_0
3906,NaN,-user unable tologin to vpn.\r\n-connected to...,awpcmsey ctdiuqwe,GRP_0
3910,NaN,-user unable tologin to vpn.\r\n-connected to...,rhwsmefo tvphyura,GRP_0
3915,NaN,-user unable tologin to vpn.\r\n-connected to...,hxripljo efzounig,GRP_0
3921,NaN,-user unable tologin to vpn.\r\n-connected to...,cziadygo veiosxby,GRP_0
3924,NaN,name:wvqgbdhm fwchqjor\nlanguage:\nbrowser:mic...,wvqgbdhm fwchqjor,GRP_0
4341,NaN,\r\n\r\nreceived from: eqmuniov.ehxkcbgj@gmail...,eqmuniov ehxkcbgj,GRP_0


In [11]:
nan_rows2

,Short description,Description,Caller,Assignment group
4395,i am locked out of skype,NaN,viyglzfo ajtfzpkb,GRP_0


In [0]:
df.loc[df['Description'].isnull(),'Description'] = df['Short description']

In [13]:
df.loc[[4395]]

,Short description,Description,Caller,Assignment group
4395,i am locked out of skype,i am locked out of skype,viyglzfo ajtfzpkb,GRP_0


In [0]:
df.loc[df['Short description'].isnull(),'Short description'] = df['Description']

In [15]:
df.loc[[3924]]

,Short description,Description,Caller,Assignment group
3924,name:wvqgbdhm fwchqjor\nlanguage:\nbrowser:mic...,name:wvqgbdhm fwchqjor\nlanguage:\nbrowser:mic...,wvqgbdhm fwchqjor,GRP_0


In [16]:
df.isnull().sum()

Short description    0
Description          0
Caller               0
Assignment group     0
dtype: int64

Preprocess text

In [0]:
# Merge 'Short description' and 'Description' columns into single column 'description'
df['description'] = (df['Short description']+df['Description']).map(str)

In [0]:
df_Tickets = df.drop(['Description'], axis=1)

In [19]:
df_Tickets.head()

,Short description,Caller,Assignment group,description
0,login issue,spxjnwir pjlcoqds,GRP_0,login issue-verified user details.(employee# &...
1,outlook,hmjdrvpb komuaywn,GRP_0,outlook\r\n\r\nreceived from: hmjdrvpb.komuayw...
2,cant log in to vpn,eylqgodm ybqkwiam,GRP_0,cant log in to vpn\r\n\r\nreceived from: eylqg...
3,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0,unable to access hr_tool pageunable to access ...
4,skype error,owlgqjme qhcozdfx,GRP_0,skype error skype error


In [0]:
df_Tickets = df_Tickets.drop(['Short description'], axis=1)

In [21]:
df_Tickets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8500 entries, 0 to 8499
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Caller            8500 non-null   object
 1   Assignment group  8500 non-null   object
 2   description       8500 non-null   object
dtypes: object(3)
memory usage: 199.3+ KB


In [22]:
df_Tickets['Assignment group'].nunique()

74

In [23]:
df_Tickets.head()

,Caller,Assignment group,description
0,spxjnwir pjlcoqds,GRP_0,login issue-verified user details.(employee# &...
1,hmjdrvpb komuaywn,GRP_0,outlook\r\n\r\nreceived from: hmjdrvpb.komuayw...
2,eylqgodm ybqkwiam,GRP_0,cant log in to vpn\r\n\r\nreceived from: eylqg...
3,xbkucsvz gcpydteq,GRP_0,unable to access hr_tool pageunable to access ...
4,owlgqjme qhcozdfx,GRP_0,skype error skype error


In [0]:
df_Tickets['description']= df_Tickets['description'].astype(str).str.replace('--@--',' ')

In [25]:
df_Tickets.head()

,Caller,Assignment group,description
0,spxjnwir pjlcoqds,GRP_0,login issue-verified user details.(employee# &...
1,hmjdrvpb komuaywn,GRP_0,outlook\r\n\r\nreceived from: hmjdrvpb.komuayw...
2,eylqgodm ybqkwiam,GRP_0,cant log in to vpn\r\n\r\nreceived from: eylqg...
3,xbkucsvz gcpydteq,GRP_0,unable to access hr_tool pageunable to access ...
4,owlgqjme qhcozdfx,GRP_0,skype error skype error


In [26]:
df_Tickets = df_Tickets.drop(['Caller'], axis=1)
df_Tickets.head()

,Assignment group,description
0,GRP_0,login issue-verified user details.(employee# &...
1,GRP_0,outlook\r\n\r\nreceived from: hmjdrvpb.komuayw...
2,GRP_0,cant log in to vpn\r\n\r\nreceived from: eylqg...
3,GRP_0,unable to access hr_tool pageunable to access ...
4,GRP_0,skype error skype error


In [0]:
df_Tickets['description'] = df_Tickets['description'].str.strip('[]')

In [0]:
df_Tickets['description'] = df_Tickets['description'].str.replace("'", " ")

In [29]:
df_Tickets.head()

,Assignment group,description
0,GRP_0,login issue-verified user details.(employee# &...
1,GRP_0,outlook\r\n\r\nreceived from: hmjdrvpb.komuayw...
2,GRP_0,cant log in to vpn\r\n\r\nreceived from: eylqg...
3,GRP_0,unable to access hr_tool pageunable to access ...
4,GRP_0,skype error skype error


In [0]:
df_Tickets.index = df_Tickets.index.astype(str)

In [31]:
df_Tickets

,Assignment group,description
0,GRP_0,login issue-verified user details.(employee# &...
1,GRP_0,outlook\r\n\r\nreceived from: hmjdrvpb.komuayw...
2,GRP_0,cant log in to vpn\r\n\r\nreceived from: eylqg...
3,GRP_0,unable to access hr_tool pageunable to access ...
4,GRP_0,skype error skype error
...,...,...
8495,GRP_29,emails not coming in from zz mail\r\n\r\nrecei...
8496,GRP_0,telephony_software issuetelephony_software issue
8497,GRP_0,vip2: windows password reset for tifpdchb pedx...
8498,GRP_62,machine nÃ£o estÃ¡ funcionandoi am unable to a...


In [0]:
df_Tickets['Descrip_less'] = df_Tickets['description'].apply(lambda x : x.rsplit(maxsplit=len(x.split())-10)[0])

In [33]:
df_Tickets1 = df_Tickets.drop(['description'], axis=1)
df_Tickets1.head()

,Assignment group,Descrip_less
0,GRP_0,login issue-verified user details.(employee# &...
1,GRP_0,outlook\r\n\r\nreceived from: hmjdrvpb.komuayw...
2,GRP_0,cant log in to vpn\r\n\r\nreceived from: eylqg...
3,GRP_0,unable
4,GRP_0,skype


In [34]:
df_new = df_Tickets1.values.tolist()
#df_Tickets1 = df_Tickets.to_dict('range')
df_new

[['GRP_0',
  'login issue-verified user details.(employee# & manager name)\r\n-checked the user'],
 ['GRP_0',
  'outlook\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail.com\r\n\r\nhello team,\r\n\r\nmy meetings/skype meetings etc'],
 ['GRP_0',
  'cant log in to vpn\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail.com\r\n\r\nhi\r\n\r\ni'],
 ['GRP_0', 'unable'],
 ['GRP_0', 'skype'],
 ['GRP_0', 'unable to log in to engineering tool and skypeunable to'],
 ['GRP_1',
  'event: critical:HostName_221.company.com the value of mountpoint threshold for /oracle/SID_37/erpdata21/ event:'],
 ['GRP_0',
  'ticket_no1550391- employment status - new non-employee [enter user s name]ticket_no1550391-'],
 ['GRP_0', 'unable to disable add ins on outlookunable to disable add'],
 ['GRP_0', 'ticket'],
 ['GRP_0',
  'engineering tool says not connected and unable to submit reportsengineering'],
 ['GRP_0',
  'hr_tool site not loading page correctlyhr_tool site not loading page'],
 ['GRP_0', 'unable to login to hr_tool

In [0]:
import json
class mydict(dict):
  def __str__(self):
    return json.dumps(self)

In [36]:
df_final =  json.dumps(df_new)
df_final

'[["GRP_0", "login issue-verified user details.(employee# & manager name)\\r\\n-checked the user"], ["GRP_0", "outlook\\r\\n\\r\\nreceived from: hmjdrvpb.komuaywn@gmail.com\\r\\n\\r\\nhello team,\\r\\n\\r\\nmy meetings/skype meetings etc"], ["GRP_0", "cant log in to vpn\\r\\n\\r\\nreceived from: eylqgodm.ybqkwiam@gmail.com\\r\\n\\r\\nhi\\r\\n\\r\\ni"], ["GRP_0", "unable"], ["GRP_0", "skype"], ["GRP_0", "unable to log in to engineering tool and skypeunable to"], ["GRP_1", "event: critical:HostName_221.company.com the value of mountpoint threshold for /oracle/SID_37/erpdata21/ event:"], ["GRP_0", "ticket_no1550391- employment status - new non-employee [enter user s name]ticket_no1550391-"], ["GRP_0", "unable to disable add ins on outlookunable to disable add"], ["GRP_0", "ticket"], ["GRP_0", "engineering tool says not connected and unable to submit reportsengineering"], ["GRP_0", "hr_tool site not loading page correctlyhr_tool site not loading page"], ["GRP_0", "unable to login to hr_too

In [0]:
#pip install spacy

In [0]:
def callback_method (matcher, doc, i, matches):
  string_id = nlp.vocab.strings[match_id] 
  match_id, start, end in matches[i]
  span = doc[start:end]  # The matched span
  print(match_id, string_id, start, end, span.text)
  

In [58]:
import spacy
from collections import Counter
from spacy.matcher import PhraseMatcher
nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)
terms = ["GRP_8", "GRP_135", "what type of outage"]
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in terms]
matcher.add("what type of outage", None, *patterns)
doc = nlp(df_final)
matches = matcher(doc)
d = []
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]  # get the unicode ID
    span = doc[start : end]  # get the matched slice of the doc
    d.append((rule_id, span.text))

print("\n".join(f'{i[0]} {i[1]} ({j})' for i,j in Counter(d).items()))

what ztype of outage GRP_8 (661)


In [40]:
import spacy
from collections import Counter
from spacy.matcher import PhraseMatcher
nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)
terms = ["GRP_0", "GRP_24", "GRP_25", "EU_tool"]
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in terms]
matcher.add("EU_tool", None, *patterns)
doc = nlp(df_final)
matches = matcher(doc)
d = []
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]  # get the unicode ID
    span = doc[start : end]  # get the matched slice of the doc
    d.append((rule_id, span.text))
print("\n".join(f'{i} ({j})' for i,j in Counter(d).items()))

('EU_tool', 'GRP_0') (3976)
('EU_tool', 'GRP_24') (289)
('EU_tool', 'GRP_25') (116)
('EU_tool', 'EU_tool') (68)


In [41]:
import spacy
from collections import Counter
from spacy.matcher import PhraseMatcher
nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)
terms = ["GRP_17", "reset password"]
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in terms]
matcher.add("reset password", None, *patterns)
matcher.add("GRP_17", None, *patterns)
doc = nlp(df_final)
matches = matcher(doc)
d = []
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]  # get the unicode ID
    span = doc[start : end]  # get the matched slice of the doc
    d.append((rule_id, span.text))
print("\n".join(f'{i} ({j})' for i,j in Counter(d).items()))

('reset password', 'GRP_17') (81)
('GRP_17', 'GRP_17') (81)
('reset password', 'reset password') (26)
('GRP_17', 'reset password') (26)


In [42]:
import spacy
from collections import Counter
from spacy.matcher import PhraseMatcher
nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)
terms = ["GRP_35", "need access"]
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in terms]
matcher.add("need access", None, *patterns)
matcher.add("GRP_35", None, *patterns)
doc = nlp(df_final)
matches = matcher(doc)
d = []
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]  # get the unicode ID
    span = doc[start : end]  # get the matched slice of the doc
    d.append((rule_id, span.text))
print("\n".join(f'{i} ({j})' for i,j in Counter(d).items()))

('need access', 'need access') (29)
('GRP_35', 'need access') (29)
('need access', 'GRP_35') (1)
('GRP_35', 'GRP_35') (1)


In [43]:
import spacy
from collections import Counter
from spacy.matcher import PhraseMatcher
nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)
terms = ["GRP_0", "windows account"]
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in terms]
matcher.add("windows account", None, *patterns)
matcher.add("GRP_0", None, *patterns)
doc = nlp(df_final)
matches = matcher(doc)
d = []
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]  # get the unicode ID
    span = doc[start : end]  # get the matched slice of the doc
    d.append((rule_id, span.text))
print("\n".join(f'{i[0]} {i[1]} ({j})' for i,j in Counter(d).items()))

windows account GRP_0 (3976)
GRP_0 GRP_0 (3976)
windows account windows account (10)
GRP_0 windows account (10)


In [0]:
doc_new=(nlp.make_doc(text) for text in df_Tickets1)

In [45]:
df_Tickets['word_count'] = df_Tickets['description'].apply(lambda x: len(str(x).split(" ")))
df_Tickets.head()

,Assignment group,description,Descrip_less,word_count
0,GRP_0,login issue-verified user details.(employee# &...,login issue-verified user details.(employee# &...,30
1,GRP_0,outlook\r\n\r\nreceived from: hmjdrvpb.komuayw...,outlook\r\n\r\nreceived from: hmjdrvpb.komuayw...,23
2,GRP_0,cant log in to vpn\r\n\r\nreceived from: eylqg...,cant log in to vpn\r\n\r\nreceived from: eylqg...,13
3,GRP_0,unable to access hr_tool pageunable to access ...,unable,9
4,GRP_0,skype error skype error,skype,5
